In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes

In [ ]:
!pip install fastapi uvicorn pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install plantuml

In [ ]:
!pip install pydantic
!pip install nest-asyncio

In [ ]:
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import FileResponse
from pydantic import BaseModel
import uuid
import os
import plantuml
from unsloth import FastLanguageModel
import torch
import uvicorn
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [ ]:
app = FastAPI() # FastAPI

In [ ]:
origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [ ]:
def get_model():
  model, tokenizer = FastLanguageModel.from_pretrained(
      model_name = 'RakhiNair/plantuml_generation_model', # trained model
      max_seq_length = 1028,
      dtype = None,
      load_in_4bit = True,
  )
#   return model,tokenizer
  return model,tokenizer


In [ ]:
model,tokenizer=get_model()

adapter_config.json:   0%|          | 0.00/732 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Unsloth 2024.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
def generate_plantuml_code(text):
    # This function interacts with the fine-tuned LLM to generate PlantUML code
    alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""
    inputs = tokenizer(
[
    alpaca_prompt.format(
        "For the given description, generate a Sequence diagram using plantuml.", # instruction
        text.strip(), # input
        "", # output
    )
], return_tensors = "pt").to("cuda")
    response = model.generate(**inputs, max_new_tokens = 512, use_cache = True)
    output = tokenizer.batch_decode(response)
    rps = output[0].split('###')
    op = rps[-1].replace('Response:','')
    return op


In [ ]:
class ScenarioInput(BaseModel):
    text: str

In [ ]:
@app.post("/generate-diagram")
async def generate_diagram(scenario: ScenarioInput):

    plantumlCode = generate_plantuml_code(scenario.text)
    image_url = plantuml.PlantUML(url='http://www.plantuml.com/plantuml/img/').get_url(plantumlCode)

    # Return the diagram image
    return {"image_url": image_url,
            "image_code":plantumlCode}


In [ ]:
!ngrok authtoken "provide your ngrok authtoken"

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
# Establish a tunnel to the local web server running on port 8000 using ngrok
ngrok_tunnel = ngrok.connect(8000)
# Print the public URL provided by ngrok, which can be used to access the local server from the internet
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()

# Run the FastAPI app using uvicorn on port 8000
uvicorn.run(app, port=8000)

INFO:     Started server process [1647]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://de54-34-30-169-12.ngrok-free.app
